# Inicialización

In [1]:
import logging

import numpy as np
import pandas as pd

import torch
import transformers

# Cargar datos

Carga los datos de texto del archivo 'imdb_reviews_small.tsv' file.

Es un archivo de valores separados por tabuladores (TSV), lo cual significa que cada uno de los campos están separados por tabuladores (en lugar de comas, como has visto en otros ejercicios de TripleTen).

In [4]:
data = pd.read_csv("/datasets/imdb_reviews_small.tsv", sep='\t')

# Tokenizador BERT

Crea el tokenizador BERT a partir de un modelo previamente entrenado que se llama 'bert-base-uncased' en transformadores. Puedes consultar rápidamente una descripción general [aquí](https://huggingface.co/transformers/pretrained_models.html), Y para obtener más detalles, puedes leer [aquí](https://huggingface.co/bert-base-uncased).

In [5]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

Hay un ejemplo de cómo obtener tokens para un solo texto dado.

Puedes usarlo para procesar todos los datos que cargaste anteriormente. Como ya hay muchos textos, y es probable que los proceses en un bucle, las longitudes mínimas/máximas de los vectores se pueden calcular de dos formas: dentro de un bucle o después de un bucle.

En el último caso, los vectores de identificadores numéricos de tokens (`ids`) y máscaras de atención (`attention_mask`) se deben almacenar en dos listas separadas. Se pueden llamar `ids_list` y `atencion_mask_list`, respectivamente. El primer caso te permite evitar la creación de esas listas, a menos que desees utilizarlas con otra finalidad, por ejemplo, para propagarlas en un modelo BERT. No se requiere en esta tarea, pero se requerirá en el proyecto.

Teniendo en cuenta lo anterior, es posible que desees combinar ambas formas para calcular las longitudes mínimas/máximas de los vectores para tokens y máscaras de atención, y conservar el resultado del tokenizador para su posterior procesamiento. Solo considera que no tiene mucho sentido mantener vectores de más de 512 elementos, ya que esta es la longitud máxima de vectores que BERT puede aceptar.

In [6]:
# textos a tokens
text = 'Es muy práctico utilizar transformadores'

# añadiendo este truco para suprimir avisos de salidas demasiado largas
# normalmente no necesitamos eso, pero en este caso queremos explorar
# cuál es la longitud máxima de ID para nuestro conjunto de reseñas
# por eso no truncamos la salida (ids) de max_length
# con los parámetros max_length=max_length y truncation=True
logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)

ids = tokenizer.encode(text.lower(), add_special_tokens=True)

# padding (agregar ceros al vector para hacer que su longitud sea igual a n)
n = 512
padded = np.array(ids[:n] + [0]*(n - len(ids)))

# creación de la máscara de atención para distinguir los tokens que nos interesan
attention_mask = np.where(padded != 0, 1, 0)

In [7]:
print(ids)

[101, 9686, 14163, 2100, 10975, 28804, 2080, 21183, 18622, 9057, 10938, 26467, 2229, 102]


In [8]:
print(padded)

[  101  9686 14163  2100 10975 28804  2080 21183 18622  9057 10938 26467
  2229   102     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [9]:
print(attention_mask)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

Escribe tu código para tokenizar los datos de texto cargados.



In [12]:
def tokenize_with_bert(texts, max_length=512):
    ids_list = []
    attention_mask_list = []

    min_tokenized_text_length = 1e7
    max_tokenized_text_length = 0

    for text in texts:

        # Suprimir advertencias sobre salidas largas
        logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)

        # Tokenizar el texto en minúsculas, agregar tokens especiales
        ids = tokenizer.encode(text.lower(), add_special_tokens=True)

        ids_len = len(ids)

        # Actualizar longitudes mínimas y máximas
        if ids_len < min_tokenized_text_length:
            min_tokenized_text_length = ids_len
        if ids_len > max_tokenized_text_length:
            max_tokenized_text_length = ids_len

        # Truncar si excede el máximo permitido por BERT
        if ids_len > max_length:
            ids = ids[:max_length]

        # Rellenar hasta max_length con ceros (padding token ID = 0 en BERT)
        padded = ids + [0] * (max_length - len(ids))

        # Crear máscara de atención: 1 para tokens reales, 0 para padding
        attention_mask = [1] * len(ids) + [0] * (max_length - len(ids))

        ids_list.append(padded)
        attention_mask_list.append(attention_mask)

    print(f'La longitud mínima de los vectores: {min_tokenized_text_length}')
    print(f'La longitud máxima de los vectores: {max_tokenized_text_length}')

    return ids_list, attention_mask_list

Ejecuta el tokenizador para todos los datos. Puede tomar un poco de tiempo como

In [13]:
ids_list, attention_mask_list = tokenize_with_bert(texts=data['review'])

Token indices sequence length is longer than the specified maximum sequence length for this model (613 > 512). Running this sequence through the model will result in indexing errors


La longitud mínima de los vectores: 18
La longitud máxima de los vectores: 3047


Veamos algunos ejemplos.

In [14]:
# cuenta elementos distintos de cero
ids_array = np.count_nonzero(np.array(ids_list), axis=1)
# almacenar el índice de la revisión que tiene la menor cantidad de tokens después de la tokenización
short_review_idx = np.argmin(np.count_nonzero(np.array(ids_list), axis=1))

In [15]:
# la reseña más corta
data['review'].iloc[short_review_idx]

'More suspenseful, more subtle, much, much more disturbing....'

In [16]:
# incorporación de la reseña más corta
ids_list[short_review_idx][:50]

[101,
 2062,
 23873,
 3993,
 1010,
 2062,
 11259,
 1010,
 2172,
 1010,
 2172,
 2062,
 14888,
 1012,
 1012,
 1012,
 1012,
 102,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [17]:
# Revisión más breve integrada basada en BERT
tokenizer.decode(ids_list[short_review_idx][:50], clean_up_tokenization_spaces=False)

'[CLS] more suspenseful , more subtle , much , much more disturbing . . . . [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'